In [29]:
import requests
import json
import pandas as pd
from functools import reduce
from pandas.io.json import json_normalize

URL = 'http://35.187.214.148:8983/solr/dbnews/select'
def get_search_result(search_words):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '" AND datetime:[2017-03-20T00:00:00Z TO 2017-04-03T23:59:59Z]', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result

In [45]:
df_moon_result = json_normalize(get_search_result('문재인'))
df_ahn_result = json_normalize(get_search_result('안철수'))
df_hong_result = json_normalize(get_search_result('홍준표'))
df_you_result = json_normalize(get_search_result('유승민'))
df_sim_result = json_normalize(get_search_result('심상정'))

In [46]:
df_moon_result['tot'] = df_moon_result['n_shares'].fillna(0) + df_moon_result['o_shares'].fillna(0) 
df_ahn_result['tot'] = df_ahn_result['n_shares'].fillna(0) + df_ahn_result['o_shares'].fillna(0) 
df_hong_result['tot'] = df_hong_result['n_shares'].fillna(0) + df_hong_result['o_shares'].fillna(0) 
df_you_result['tot'] = df_you_result['n_shares'].fillna(0) + df_you_result['o_shares'].fillna(0) 
df_sim_result['tot'] = df_sim_result['n_shares'].fillna(0) + df_sim_result['o_shares'].fillna(0) 

In [47]:
tot = len(df_moon_result) + len(df_ahn_result) + len(df_hong_result) + len(df_you_result)+ len(df_sim_result)

# 후보자별 공유된 뉴스 비율

후보자별 공유된 뉴스 갯수/전체 공유된 뉴스 합

In [66]:
print("문재인 : %0.2f" %(len(df_moon_result)*100/tot))
print("안철수 : %0.2f" %(len(df_ahn_result)*100/tot))
print("홍준표 : %0.2f" %(len(df_hong_result)*100/tot))
print("유승민 : %0.2f" %(len(df_you_result)*100/tot))
print("심상정 : %0.2f" %(len(df_sim_result)*100/tot))

문재인 : 37.73
안철수 : 16.69
홍준표 : 21.70
유승민 : 19.70
심상정 : 4.17


In [67]:
shared_tot = df_moon_result['tot'].sum()+df_ahn_result['tot'].sum()+df_you_result['tot'].sum()+df_hong_result['tot'].sum()+df_sim_result['tot'].sum()

# 후보자별 뉴스 공유수 비율

후보자별 뉴스 공유수 합/전체 뉴스 공유 수

In [68]:
print("문재인 : %0.2f" %(df_moon_result['tot'].sum()*100/shared_tot))
print("안철수 : %0.2f" %(df_ahn_result['tot'].sum()*100/shared_tot))
print("홍준표 : %0.2f" %(df_hong_result['tot'].sum()*100/shared_tot))
print("유승민 : %0.2f" %(df_you_result['tot'].sum()*100/shared_tot))
print("심상정 : %0.2f" %(df_sim_result['tot'].sum()*100/shared_tot))

문재인 : 64.51
안철수 : 17.66
홍준표 : 13.37
유승민 : 3.87
심상정 : 0.59


In [75]:
moon_by_media = df_moon_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')
ahn_by_media = df_ahn_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')
hong_by_media = df_hong_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')
you_by_media = df_you_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')
sim_by_media = df_sim_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')

# 문재인 후보 언론사별 공유수

In [76]:
moon_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news.articles,NaN,NaN,0.0
news2016.chosun,39.0,25.0,64.0
news2016.donga,78.0,6.0,84.0
news2016.edaily,52.0,17.0,69.0
news2016.hani,NaN,10.0,10.0
news2016.joins,226.0,41.0,267.0
news2016.jtbc,NaN,2.0,2.0
news2016.kbs,80.0,5.0,85.0
news2016.kh,NaN,4361.0,4361.0


# 안철수 후보 언론사별 공유수

In [77]:
ahn_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news.articles,NaN,NaN,0.0
news2016.chosun,NaN,960.0,960.0
news2016.donga,9.0,NaN,9.0
news2016.edaily,4.0,11.0,15.0
news2016.hani,NaN,2.0,2.0
news2016.joins,9.0,302.0,311.0
news2016.jtbc,NaN,3.0,3.0
news2016.kbs,NaN,17.0,17.0
news2016.kh,6.0,9.0,15.0


# 홍준표 후보 언론사별 공유수

In [78]:
hong_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news.articles,NaN,NaN,0.0
news2016.chosun,2.0,32.0,34.0
news2016.donga,43.0,1.0,44.0
news2016.edaily,42.0,NaN,42.0
news2016.hani,NaN,68.0,68.0
news2016.joins,8.0,227.0,235.0
news2016.jtbc,NaN,6.0,6.0
news2016.kbs,5.0,1.0,6.0
news2016.kh,1.0,3.0,4.0


# 유승민 후보 언론사별 공유수

In [79]:
you_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news.articles,NaN,NaN,0.0
news2016.chosun,24.0,21.0,45.0
news2016.edaily,7.0,2.0,9.0
news2016.hani,NaN,69.0,69.0
news2016.joins,14.0,32.0,46.0
news2016.jtbc,NaN,6.0,6.0
news2016.kbs,4.0,NaN,4.0
news2016.kh,5.0,1.0,6.0
news2016.mbc,NaN,1.0,1.0


# 심상정 후보 언론사별 공유수

In [80]:
sim_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news.articles,NaN,NaN,0.0
news2016.chosun,NaN,2.0,2.0
news2016.donga,NaN,4.0,4.0
news2016.kbs,1.0,NaN,1.0
news2016.kh,2.0,1.0,3.0
news2016.nocut,11.0,NaN,11.0
news2016.ohmynews,NaN,3.0,3.0
news2016.yonhap,32.0,NaN,32.0
